In [ ]:
!pip install playwright pandas -q
!playwright install chromium
!apt-get install libxcomposite1 libgtk-3-0 libatk1.0-0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 MB 11.4 MB/s eta 0:00:00
(node:485) [DEP0169] DeprecationWarning: `url.parse()` behavior is not standardized and prone to errors that have security implications. Use the WHATWG URL API instead. CVEs are not issued for `url.parse()` vulnerabilities.
(Use `node --trace-deprecation ...` to show where the warning was created)
164.7 MiB [] 0% 0.0s164.7 MiB [] 0% 45.8s164.7 MiB [] 0% 14.8s164.7 MiB [] 0% 13.6s164.7 MiB [] 0% 8.9s164.7 MiB [] 1% 5.2s164.7 MiB [] 2% 4.1s164.7 MiB [] 3% 3.4s164.7 MiB [] 3% 3.6s164.7 MiB [] 3% 4.1s164.7 MiB [] 3% 4.6s164.7 MiB [] 3% 5.1s164.7 MiB [] 3% 5.5s164.7 MiB [] 3% 5.9s164.7 MiB [] 3% 6.4s164.7 MiB [] 3% 6.8s164.7 MiB [] 3% 7.2s164.7 MiB [] 3% 7.7s164.7 MiB [] 3% 8.1s164.7 MiB [] 3% 8.5s164.7 MiB [] 3% 8.7s164.7 MiB [] 4% 8.3s164.7 MiB [] 4% 7.9s164.7 MiB [] 5% 7.3s164.7 MiB [] 5% 7.0s164.7 MiB [] 6% 6.6s164.7 MiB [] 6% 6.2s164.7 MiB [] 7% 5.6s164.7 MiB [] 8% 5.1s164.7 MiB [] 9% 4.7s164.7 MiB [] 10% 4.

In [ ]:
import asyncio
import json
import csv
from playwright.async_api import async_playwright
import pandas as pd

In [ ]:
class RajaSahibScraper:
    def __init__(self):
        self.base_url = "https://rajasahibfresh.pk"
        self.products = []
        self.seen_product_ids = set()

    async def handle_location_modal(self, page):
        try:
            await page.wait_for_selector('input[placeholder*="City"]', timeout=5000)
            await page.fill('input[placeholder*="City"]', 'Lahore')
            await page.keyboard.press('Enter')
            await page.wait_for_timeout(1000)

            await page.click('button:has-text("Select")')
            await page.wait_for_timeout(2000)
        except:
            pass

    async def scroll_and_load_all(self, page):
        previous_height = 0
        attempts = 0
        max_attempts = 10

        while attempts < max_attempts:
            current_height = await page.evaluate('document.body.scrollHeight')

            if current_height == previous_height:
                break

            await page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
            await page.wait_for_timeout(2000)
            previous_height = current_height
            attempts += 1

    async def extract_products_from_page(self, page, category_url):
        products = await page.evaluate('''(categoryUrl) => {
            const productCards = document.querySelectorAll('div[class*="product-item_product_item__"]');
            const results = [];

            productCards.forEach(card => {
                try {
                    const nameElement = card.querySelector('h4');
                    const priceElement = card.querySelector('[class*="product-item_product_item_price_label__"] span:last-child');
                    const oldPriceElement = card.querySelector('[class*="product-item_product_item_old_price_label__"]');
                    const imgElement = card.querySelector('[class*="product-item_product_item_image_container__"] img');
                    const discountElement = card.querySelector('[class*="product-item_product_item_image_discount__"]');

                    const name = nameElement ? nameElement.innerText.trim() : '';
                    const priceText = priceElement ? priceElement.innerText : '';
                    const oldPriceText = oldPriceElement ? oldPriceElement.innerText : '';
                    const img = imgElement ? imgElement.src : '';
                    const productSlug = imgElement ? imgElement.title : '';
                    const discountText = discountElement ? discountElement.innerText : '';

                    const priceMatch = priceText.match(/Rs[.\\s]*(\\d+[\\d,]*)/);
                    const price = priceMatch ? priceMatch[1].replace(/,/g, '') : '';

                    const oldPriceMatch = oldPriceText.match(/Rs[.\\s]*(\\d+[\\d,]*)/);
                    const oldPrice = oldPriceMatch ? oldPriceMatch[1].replace(/,/g, '') : '';

                    const discountMatch = discountText.match(/(\\d+)%/);
                    const discount = discountMatch ? discountMatch[1] : '';

                    const productId = card.id || '';

                    if (name && price) {
                        results.push({
                            id: productId,
                            name: name,
                            price: price,
                            original_price: oldPrice,
                            image_url: img,
                            discount: discount,
                            product_slug: productSlug,
                            category_url: categoryUrl
                        });
                    }
                } catch (e) {
                    console.log('Error parsing product card:', e);
                }
            });

            return results;
        }''', category_url)

        return products

    async def get_all_subcategory_urls(self, page):
        await page.goto(self.base_url, wait_until='domcontentloaded')
        await self.handle_location_modal(page)

        try:
            await page.click('button:has-text("Categories")')
            await page.wait_for_timeout(2000)
        except:
            pass

        parent_categories = await page.evaluate('''() => {
            const links = document.querySelectorAll('a[href*="/catalog/"]');
            return Array.from(new Set(Array.from(links).map(link => link.href)));
        }''')

        all_urls = set()

        for parent_url in parent_categories:
            try:
                print(f"Checking: {parent_url.split('/')[-1]}")
                await page.goto(parent_url, wait_until='domcontentloaded', timeout=30000)
                await self.handle_location_modal(page)
                await page.wait_for_timeout(1000)

                subcategories = await page.evaluate('''() => {
                    const links = document.querySelectorAll('a[href*="/catalog/"]');
                    return Array.from(new Set(Array.from(links).map(link => link.href)));
                }''')

                product_count = await page.evaluate('''() => {
                    return document.querySelectorAll('div[class*="product-item_product_item__"]').length;
                }''')

                if product_count > 0:
                    all_urls.add(parent_url)
                    print(f"  → Has {product_count} products")

                for sub_url in subcategories:
                    if sub_url != parent_url:
                        all_urls.add(sub_url)

            except Exception as e:
                print(f"Error checking {parent_url}: {e}")
                continue

        return list(all_urls)

    async def scrape_category(self, page, category_url):
        print(f"\nScraping: {category_url.split('/')[-1]}")

        try:
            await page.goto(category_url, wait_until='domcontentloaded', timeout=30000)
            await self.handle_location_modal(page)
            await self.scroll_and_load_all(page)

            products = await self.extract_products_from_page(page, category_url)

            for product in products:
                product_id = product.get('id', '')

                if product_id and product_id not in self.seen_product_ids:
                    self.seen_product_ids.add(product_id)

                    product_slug = product.get('product_slug', '')
                    product_url = f"https://rajasahibfresh.pk/product/{product_slug}-{product_id}" if product_slug and product_id else category_url

                    self.products.append({
                        'name': product['name'],
                        'price': product['price'],
                        'original_price': product.get('original_price', ''),
                        'discount': product.get('discount', ''),
                        'image_url': product['image_url'],
                        'url': product_url
                    })

            print(f"  Found {len(products)} products | Total unique: {len(self.seen_product_ids)}")

        except Exception as e:
            print(f"  Error: {e}")

    async def scrape_all(self):
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            context = await browser.new_context()
            page = await context.new_page()

            print("Discovering all categories...")
            category_urls = await self.get_all_subcategory_urls(page)
            print(f"\nFound {len(category_urls)} categories/subcategories\n")
            print("="*50)

            for i, url in enumerate(category_urls, 1):
                print(f"[{i}/{len(category_urls)}]", end=" ")
                await self.scrape_category(page, url)

            await browser.close()

        return self.products

    def save_to_csv(self, filename='raja_sahib_products.csv'):
        df = pd.DataFrame(self.products)
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"\nSaved {len(self.products)} products to {filename}")
        return df

In [ ]:
scraper = RajaSahibScraper()
products = await scraper.scrape_all()
df = scraper.save_to_csv()

print(f"\n{'='*50}")
print(f"SCRAPING COMPLETE!")
print(f"Total products scraped: {len(products)}")
print(f"{'='*50}")

Discovering all categories...
Checking: mega-discounts-32350
  → Has 240 products
Checking: beverages-31039
Checking: breakfast-31040
Checking: hair-care-31050
Checking: kitchen--home-appliances-32246
Checking: electronic-accessories-32245
Checking: makeup-32244
Checking: hand--foot-care-31067
Checking: women--adult-care-31057
Checking: skin-care-31056
Checking: shoe-care-31055
  → Has 44 products
Checking: pet-care-31054
Checking: mens-care-31053
Checking: meat-31052
Checking: health--wellness-31051
Checking: baby-foods--diapers-31037
Checking: fruits-and-vegetables-31049
  → Has 36 products
Checking: frozen-31048
Checking: fragrances-31047
Checking: dental-care-31046
Checking: dairy-31045
Checking: cooking-essentials-31044
Checking: cleaning--household-31043
Checking: chips-and-snacks-31042
Checking: car-care-31041
Checking: bath--body-31038

Found 166 categories/subcategories

[1/166] 
Scraping: kajal--mascara-60658
  Found 12 products | Total unique: 12
[2/166] 
Scraping: nail-pain

In [ ]:
print("\nFirst 10 products:")
df.head(10)


First 10 products:


,name,price,original_price,discount,image_url,url
0,Maybelline New York Mascara Colossal Waterproo...,1799,,,https://em-cdn.eatmubarak.pk/55545/gallery/690...,https://rajasahibfresh.pk/product/Maybelline N...
1,Essence Thick & Wow! Fixing Brow Mascara 02 As...,1949,,,https://em-cdn.eatmubarak.pk/55545/gallery/405...,https://rajasahibfresh.pk/product/Essence Thic...
2,Gabrini Eye Brow Mascara Dark Brown-6ml,1076,1345,20,https://em-cdn.eatmubarak.pk/55545/gallery/869...,https://rajasahibfresh.pk/product/Gabrini Eye ...
3,Gabrini Eye Brow Mascara Brown-6ml,1076,1345,20,https://em-cdn.eatmubarak.pk/55545/gallery/869...,https://rajasahibfresh.pk/product/Gabrini Eye ...
4,Olivia Kajal Stick-1.6g,60,,,https://em-cdn.eatmubarak.pk/55545/gallery/858...,https://rajasahibfresh.pk/product/Olivia Kajal...
5,Maybelline The Colossal Kajal Argan Oil Pencil...,999,,,https://em-cdn.eatmubarak.pk/55545/gallery/360...,https://rajasahibfresh.pk/product/Maybelline T...
6,Maybelline Volume Express Colossal Mascara 100...,1749,,,https://em-cdn.eatmubarak.pk/55545/gallery/300...,https://rajasahibfresh.pk/product/Maybelline V...
7,Maybelline Falsies Lash Lift Mascara 01 Very B...,3399,,,https://em-cdn.eatmubarak.pk/55545/gallery/690...,https://rajasahibfresh.pk/product/Maybelline F...
8,Rimmel London Scandaleyes Waterproof Kohl Kaja...,1199,1599,25,https://em-cdn.eatmubarak.pk/55545/gallery/361...,https://rajasahibfresh.pk/product/Rimmel Londo...
9,Rimmel London Scandaleyes Ultimate Boost Primer,1087,1450,25,https://em-cdn.eatmubarak.pk/55545/gallery/361...,https://rajasahibfresh.pk/product/Rimmel Londo...


In [ ]:
print("\nData Summary:")
print(f"Total Products: {len(df)}")
print(f"Products with Discounts: {len(df[df['discount'] != ''])}")
print(f"Average Discount: {df[df['discount'] != '']['discount'].astype(float).mean():.1f}%")
print(f"\nPrice Range: Rs.{df['price'].astype(float).min():.0f} - Rs.{df['price'].astype(float).max():.0f}")


Data Summary:
Total Products: 7961
Products with Discounts: 3269
Average Discount: 7.2%

Price Range: Rs.0 - Rs.24741


In [ ]:
df.to_csv("data.csv", index=False)